In [1296]:
import regex as re
from pathlib import Path
import spacy
from spacy import displacy
import pandas as pd
import numpy as np
import coreferee
from sentence_transformers import SentenceTransformer, util
import getopt
import sys
import pandas as pd
from datasets import load_dataset
import json 
import regex as re
from tqdm import tqdm
import nltk




This is the universal dependency for English: https://universaldependencies.org/en/dep/index.html

### Loading Model

In [1297]:
nlp = spacy.load('en_core_web_md')
nlp.add_pipe("merge_entities")
nlp.add_pipe("merge_noun_chunks")
nlp.add_pipe('coreferee')
model = SentenceTransformer("all-MiniLM-L6-v2")

# doc = """
#     Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the golden anniversary with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as Super Bowl L), so that the logo could prominently feature the Arabic numerals 50.
# """
# doc = "On May 21, 2013, NFL owners at their spring meetings in Boston voted and awarded the game to Levi's Stadium. The $1.2 billion stadium opened in 2014. It is the first Super Bowl held in the San Francisco Bay Area since Super Bowl XIX in 1985, and the first in California since Super Bowl XXXVII took place in San Diego in 2003."
# doc = "Teachers are required to be registered with the Teaching Council; under Section 30 of the Teaching Council Act 2001, a person employed in any capacity in a recognised teaching post - who is not registered with the Teaching Council - may not be paid from Oireachtas funds."
doc = "The length of the Rhine is conventionally measured in ""Rhine-kilometers"" (Rheinkilometer), a scale introduced in 1939 which runs from the Old Rhine Bridge at Constance (0 km) to Hoek van Holland (1036.20 km). The river length is significantly shortened from the river's natural course due to number of canalisation projects completed in the 19th and 20th century.[note 7] The ""total length of the Rhine"", to the inclusion of Lake Constance and the Alpine Rhine is more difficult to measure objectively; it was cited as 1,232 kilometres (766 miles) by the Dutch Rijkswaterstaat in 2010.[note 1]."

c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### resolve coreferences

In [1298]:

def resolve_coreference(text):
    doc = nlp(text)
    doc_list = list(doc)
    doc._.coref_chains.print()
    resolving_indecies = []
    for _,item in enumerate(doc._.coref_chains):
        resolving_indecies.extend(item)
        
    for word in resolving_indecies:
        new_word = ""
        for index in word:
            if doc[index]._.coref_chains.resolve(doc[index]) is not None:
                temp = []
                for item in doc._.coref_chains.resolve(doc[index]):
                    temp.append(str(item))
                new_word = ", ".join(temp)
            
                doc_list[index] = new_word

    final_doc = []
    for item in doc_list:
        final_doc.append(str(item))
    return " ".join(final_doc)
        

### Preprocess the document

In [1299]:
def preprocess_context(doc):
    text = doc.strip()
    # remove any symbols that are not needed but keep the full stop, comma and space
    # text = re.sub(r'[^\w\s.,]', '', text)
    text.replace(".", ",")
    resolved_text = resolve_coreference(text)
    resolved_text = resolved_text.strip()
    resolved_text = resolved_text.replace("  ", " ").replace(" ,", ",").replace(";","").replace(" .", ".").replace("\n", "")
    return resolved_text

In [1300]:
resolved_doc = preprocess_context(doc)
cleaned_doc = nlp(resolved_doc)
sentences = [one_sentence.text.strip() for one_sentence in cleaned_doc.sents]

0: the Rhine(2), the Rhine(48)


In [1301]:
sentences

['The length of the Rhine is conventionally measured in Rhine-kilometers (Rheinkilometer ), a scale introduced in 1939 which runs from the Old Rhine Bridge at Constance ( 0 km ) to Hoek van Holland ( 1036.20 km ).',
 "The river length is significantly shortened from the river's natural course due to number of canalisation projects completed in the 19th and 20th century.[note 7 ] The total length of the Rhine, to the inclusion of Lake Constance and the Alpine Rhine is more difficult to measure objectively  it was cited as 1,232 kilometres ( 766 miles ) by the Dutch Rijkswaterstaat in 2010.[note 1 ]."]

In [1302]:
# get all verbs in the sentences
verbs = set()
for sentence in sentences:
    doc = nlp(sentence)
    for token in doc:
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            verbs.add(token.lemma_)
            
verbs
# verbs2 = ["earn"]
# # loop on sentences and get the sentences that have verbs woth lemmas in those [earn, have, suspend]
# sentences_with_verbs = []
# for sentence in sentences:
#     doc = nlp(sentence)
#     for token in doc:
#         if token.lemma_ in verbs2:
#             sentences_with_verbs.append(sentence)
#             break
        
# sentences_with_verbs
# [earn, have, suspend]


{'be', 'cite', 'complete', 'introduce', 'measure', 'run', 'shorten'}

In [1303]:
excludesPerQuestionType = {
    "when": "Times",
    "where": "Locations",
    "who": "Subject",
    "what": "Objects",
    "how": "States"
}

In [1304]:
question = "When was the measurement of the Rhine introduced?"
# answer = "He talked to him to secure the account."
# answer = "The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title."
# answer = "As this was the 50th Super Bowl, the league emphasized the golden anniversary with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals ( under which the game would have been known as Super Bowl L ), so that the logo could prominently feature the Arabic numerals 50."
# answer = "Aly said yesterday that Ziad was good today."
# answer = "Politics: U.N. Secretary General Ban Ki-moon; American political leaders John Hancock, John Adams, John Quincy Adams, Rutherford B. Hayes, Theodore Roosevelt, Franklin D. Roosevelt, John F. Kennedy, Al Gore, George W. Bush and Barack Obama; Chilean President Sebastián Piñera; Colombian President Juan Manuel Santos; Costa Rican President José María Figueres; Mexican Presidents Felipe Calderón, Carlos Salinas de Gortari and Miguel de la Madrid; Mongolian President Tsakhiagiin Elbegdorj; Peruvian President Alejandro Toledo; Taiwanese President Ma Ying-jeou; Canadian Governor General David Lloyd Johnston; Indian Member of Parliament Jayant Sinha; Albanian Prime Minister Fan S. Noli; Canadian Prime Ministers Mackenzie King and Pierre Trudeau; Greek Prime Minister Antonis Samaras; Israeli Prime Minister Benjamin Netanyahu; former Pakistani Prime Minister Benazir Bhutto; U. S. Secretary of Housing and Urban Development Shaun Donovan; Canadian political leader Michael Ignatieff; Pakistani Members of Provincial Assembly Murtaza Bhutto and Sanam Bhutto; Bangladesh Minister of Finance Abul Maal Abdul Muhith; President of Puntland Abdiweli Mohamed Ali; U.S. Ambassador to the European Union Anthony Luzzatto Gardner."
answer =  'The length of the Rhine is conventionally measured in Rhine-kilometers (Rheinkilometer ), a scale introduced in 1939 which runs from the Old Rhine Bridge at Constance ( 0 km ) to Hoek van Holland ( 1036.20 km ).'
# answer = 'a scale introduced in 1939.'
ans_nlp = nlp(answer)
splitted_question = question.split(" ")
question_type = splitted_question[0].lower()
question_nlp = nlp(question)
if question_nlp[0].ent_type_ == "DATE":
    question_type = "when"

# if len(question_nlp) <= 2:
#     nlp = spacy.load('en_core_web_md')
#     nlp.add_pipe('coreferee')
#     question_nlp = nlp(question)
    

In [1305]:
for token in question_nlp:
    print(token.text, '-',token.pos_,'-', token.dep_,'-', token.ent_type_)

    
displacy.render(question_nlp, style="dep")

When - SCONJ - advmod - 
was - AUX - auxpass - 
the measurement - NOUN - nsubj - 
of - ADP - prep - 
the Rhine - PROPN - pobj - LOC
introduced - VERB - ROOT - 
? - PUNCT - punct - 


In [1306]:
for token in ans_nlp:
    
    print(token.text, '-',token.pos_,'-', token.dep_,'-', token.ent_type_)
        

displacy.render(ans_nlp, style="dep")

The length - NOUN - nsubjpass - 
of - ADP - prep - 
the Rhine - PROPN - pobj - LOC
is - AUX - auxpass - 
conventionally - ADV - advmod - 
measured - VERB - ROOT - 
in - ADP - prep - 
Rhine-kilometers (Rheinkilometer - NOUN - pobj - PERSON
) - PUNCT - punct - 
, - PUNCT - punct - 
a - DET - det - 
scale - NOUN - npadvmod - 
introduced - VERB - acl - 
in - ADP - prep - 
1939 - NUM - pobj - DATE
which - PRON - nsubj - 
runs - VERB - advcl - 
from - ADP - prep - 
the Old Rhine Bridge - PROPN - pobj - ORG
at - ADP - prep - 
Constance - PROPN - pobj - 
( - PUNCT - punct - 
0 km - NOUN - appos - QUANTITY
) - PUNCT - punct - 
to - ADP - prep - 
Hoek van Holland - PROPN - pobj - PERSON
( - PUNCT - punct - 
1036.20 km - NOUN - appos - QUANTITY
) - PUNCT - punct - 
. - PUNCT - punct - 


### Extracting the entities

In [1307]:
def extract_subjects(sentence):
    subjects = {}
    verbIdx = 0
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX" or token.dep_ == "ROOT":
            verbIdx += 1
            subjectFlag = False
            verb = token
            for child in token.children:
                if child.dep_ in ("nsubj", "csubj"):
                    subtree_tokens = [str(t) for t in child.subtree]
                    subjects[token] = (" ".join(subtree_tokens), verbIdx)
                    subjectFlag = True
                elif child.dep_ == "nsubjpass":
                    for child in verb.children:
                        if child.dep_ == "agent" and len(list(child.children)) > 0:
                            subject = [str(t) for t in list(child.children)[0].subtree]
                            subject = " ".join(subject)
                            break
                        else:
                            subject = "Unknown"
                    subjects[verb] = (subject, verbIdx)
                    subjectFlag = True
            if not subjectFlag:  # didn't find a normal subject
                if token.dep_ in ("relcl", "acl"):
                    subject = str(token.head)
                    subjects[token] = (subject, verbIdx)  # should get the subtree of the subject
                elif token.dep_ in ("advcl", "conj"):
                    verb = token.head
                    
                    if verb in subjects:
                        subjects[token] = (subjects[verb][0], verbIdx)
                    else:
                        subjects[token] = ("Unknown", verbIdx)  # replace "Unknown" with a suitable default
                elif token.dep_ == "xcomp":
                    verb = token.head
                    if verb in subjects:
                        subjects[token] = (subjects[verb][0], verbIdx)
                    else:
                        subjects[token] = ("Unknown", verbIdx)
                    for child in verb.subtree:
                        if child.dep_ in ("dobj", "dative", "pobj"):
                            subtree_tokens = [str(t) for t in child.subtree]
                            subjects[token] = (" ".join(subtree_tokens), verbIdx)
                            break
                else:
                    subjects[token] = ("Unknown", verbIdx)
                                        
    # (subject, verbIdx, verb)
    return [(v[0], k, v[1]) for k, v in subjects.items()]             
                            
def extract_objects(sentence):
    objects = []
    verbIdx = 0
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX" or token.dep_ == "ROOT":
            verbIdx += 1
            for child in token.children:
                if child.dep_ in ("dobj", "dative", "attr", "oprd", "acomp","ccomp", "xcomp", "nsubjpass"):
                    subtree_tokens = [str(t) for t in child.subtree]
                    objects.append((" ".join(subtree_tokens), token, verbIdx))
                    
    return objects

def extract_state(sentence):
    states = []
    verbIdx = 0
    for token in sentence:
        if token.pos_ =="VERB" or token.pos_ == "AUX":
            verbIdx += 1
            for child in token.children:
                if child.dep_ == "prep":
                    subtree_tokens = [str(t) for t in child.subtree]
                    states.append(((" ".join(subtree_tokens), token, verbIdx)))
    return states

# def extract_time(sentence):
#     times = {}
#     verbIdx = 0
#     for token in sentence:
#         if token.pos_ == "VERB" or token.pos_ == "AUX" or token.dep_ == "ROOT":
#             verbIdx += 1
#             for child in token.subtree:
#                 if child.ent_type_ == "DATE" or child.ent_type_ == "TIME":
#                     times[child.text] = (token, verbIdx)
                    
#     return [(k, v[0], v[1]) for k, v in times.items()]

def extract_time(sentence):
    times = {}
    verbIdx = 0
    year_pattern = re.compile(r'\b\d{4}\b')  # matches any four-digit number
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX" or token.dep_ == "ROOT":
            verbIdx += 1
            for child in token.subtree:
                if child.ent_type_ == "DATE" or child.ent_type_ == "TIME":
                    times[child.text] = (token, verbIdx)
                elif year_pattern.search(child.text):
                    year = year_pattern.search(child.text).group()
                    times[year] = (token, verbIdx)
    return [(k, v[0], v[1]) for k, v in times.items()]

def extract_location(sentence):
    locations = {}
    verbIdx = 0
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX" or token.dep_ == "ROOT":
            verbIdx += 1
            for child in token.subtree:
                if child.ent_type_ in ("GPE", "LOC", "FAC"):
                    locations[child.text] = (token, verbIdx)
                    
    return [(k, v[0], v[1]) for k, v in locations.items()]
                    

def extract_facts(sentence):
    sentence = nlp(sentence)
    states = extract_state(sentence)
    subjects = extract_subjects(sentence)
    objects = extract_objects(sentence)
    times = extract_time(sentence)
    locations = extract_location(sentence)
    print("Subjects: ",subjects, objects)
    print("states: " ,states, "times: ", times,"locations: ", locations)
    
    facts = pd.DataFrame(columns=["Subject", "Relation", "verbIdx", "Objects", "States", "Times", "Locations"])
    
    for subject in subjects: #(Aly, is, 1), (Ziad,is, 2) 
        currentSubject = subject[0]
        verb = subject[1].lemma_
        verbIdx = subject[2]
        mask = (facts['Subject'] != currentSubject) | (facts['Relation'] != verb)
        if mask.all():
            new_row = pd.DataFrame([{"Subject": currentSubject, "Relation": verb, "verbIdx": verbIdx, "Objects": [], "States": [], "Times": [], "Locations": []}])
            facts = pd.concat([facts, new_row], ignore_index=True)

    for obj in objects: #(happy, is, 1), (good, is, 2)
        currentObj = obj[0]
        verb = obj[1].lemma_
        verbIdx = obj[2]
        mask = (facts['Relation'] == verb) & (facts['verbIdx'] == verbIdx)
        if mask.any():
            oldObjects = list(facts.loc[mask, "Objects"].values[0])
            oldObjects.append(currentObj)
            for idx in facts.loc[mask].index:
                facts.at[idx, "Objects"] = oldObjects
            
    for state in states:
        currentState = state[0]
        verb = state[1].lemma_
        verbIdx = state[2]
        mask = (facts['Relation'] == verb) & (facts['verbIdx'] == verbIdx)
        if mask.any():
            oldStates = list(facts.loc[mask, "States"].values[0])
            oldStates.append(currentState)
            for idx in facts.loc[mask].index:
                facts.at[idx, "States"] = oldStates
            
    for time in times:
        currentTime = time[0]
        verb = time[1].lemma_
        verbIdx = time[2]
        mask = (facts['Relation'] == verb) & (facts['verbIdx'] == verbIdx)
        if mask.any():
            oldTimes = list(facts.loc[mask, "Times"].values[0])
            oldTimes.append(currentTime)
            for idx in facts.loc[mask].index:
                facts.at[idx, "Times"] = oldTimes
            
    for location in locations:
        currentLocation = location[0]
        verb = location[1].lemma_
        verbIdx = location[2]
        mask = (facts['Relation'] == verb) & (facts['verbIdx'] == verbIdx)
        if mask.any():
            oldLocations = list(facts.loc[mask, "Locations"].values[0])
            oldLocations.append(currentLocation)
            for idx in facts.loc[mask].index:
                facts.at[idx, "Locations"] = oldLocations
            
    facts = facts.drop(columns=["verbIdx"])
    return facts

In [1308]:
extract_facts(answer)

Subjects:  [('Unknown', is, 1), ('Unknown', measured, 2), ('scale', introduced, 3), ('which', runs, 4)] [('The length of the Rhine', measured, 2)]
states:  [('in Rhine-kilometers (Rheinkilometer )', measured, 2), ('in 1939', introduced, 3), ('from the Old Rhine Bridge at Constance ( 0 km', runs, 4), ('to Hoek van Holland ( 1036.20 km )', runs, 4)] times:  [('1939', introduced, 3), ('1036', runs, 4)] locations:  [('the Rhine', measured, 2)]


,Subject,Relation,Objects,States,Times,Locations
0,Unknown,be,[],[],[],[]
1,Unknown,measure,[The length of the Rhine],[in Rhine-kilometers (Rheinkilometer )],[],[the Rhine]
2,scale,introduce,[],[in 1939],[1939],[]
3,which,run,[],[from the Old Rhine Bridge at Constance ( 0 km...,[1036],[]


In [1309]:
questionDF = extract_facts(question)
#check the length of the questionDF
if len(questionDF) == 1:
    questionDF["Subject"] = question_nlp.text
    questionDF["Relation"] = question_nlp.text
    questionDF["Objects"] = question_nlp.text
    questionDF["States"] = question_nlp.text
    questionDF["Times"] = question_nlp.text
    questionDF["Locations"] = question_nlp.text
questionDF

Subjects:  [('Unknown', was, 1), ('the measurement of the Rhine', introduced, 2)] []
states:  [] times:  [] locations:  [('the Rhine', introduced, 2)]


,Subject,Relation,Objects,States,Times,Locations
0,Unknown,be,[],[],[],[]
1,the measurement of the Rhine,introduce,[],[],[],[the Rhine]


In [1310]:
def join_sentences_facts(sentences):
    all_facts = pd.DataFrame(columns=["Subject", "Relation", "Objects", "States", "Times", "Locations"])
    for sentence in sentences:
        facts = extract_facts(sentence)
        all_facts = pd.concat([all_facts, facts])
    all_facts = all_facts.groupby(["Subject", "Relation"], as_index=False).agg({
        "Objects": lambda x: [item for sublist in x for item in sublist],
        "States": lambda x: [item for sublist in x for item in sublist],
        "Times": lambda x: [item for sublist in x for item in sublist],
        "Locations": lambda x: [item for sublist in x for item in sublist]
    })
    return all_facts

In [1311]:
factsDF = join_sentences_facts(sentences)

Subjects:  [('Unknown', is, 1), ('Unknown', measured, 2), ('scale', introduced, 3), ('which', runs, 4)] [('The length of the Rhine', measured, 2)]
states:  [('in Rhine-kilometers (Rheinkilometer )', measured, 2), ('in 1939', introduced, 3), ('from the Old Rhine Bridge at Constance ( 0 km', runs, 4), ('to Hoek van Holland ( 1036.20 km )', runs, 4)] times:  [('1939', introduced, 3), ('1036', runs, 4)] locations:  [('the Rhine', measured, 2)]
Subjects:  [('Unknown', is, 1), ('Unknown', shortened, 2), ('canalisation projects', completed, 3), ('the Alpine Rhine', is, 4), ('the Rhine', measure, 5), ('Unknown', was, 6), ('the Dutch Rijkswaterstaat', cited, 7)] [('The river length', shortened, 2), ('more difficult', is, 4), ('to measure objectively  ', is, 4), ('it', cited, 7)]
states:  [("from the river's natural course", shortened, 2), ('due to number of canalisation projects completed in the 19th and 20th century.[note 7', shortened, 2), ('in the 19th and 20th century.[note 7', completed, 3

In [1312]:
factsDF

,Subject,Relation,Objects,States,Times,Locations
0,Unknown,be,[],[],[],[]
1,Unknown,measure,[The length of the Rhine],[in Rhine-kilometers (Rheinkilometer )],[],[the Rhine]
2,Unknown,shorten,[The river length],"[from the river's natural course, due to numbe...",[],[]
3,canalisation projects,complete,[],[in the 19th and 20th century.[note 7],[],[]
4,scale,introduce,[],[in 1939],[1939],[]
5,the Alpine Rhine,be,"[more difficult, to measure objectively ]",[],[],"[the Rhine, the Alpine Rhine]"
6,the Dutch Rijkswaterstaat,cite,[it],"[as 1,232 kilometres ( 766 miles ), in 2010.[n...",[2010],[]
7,the Rhine,measure,[],[],[],[]
8,which,run,[],[from the Old Rhine Bridge at Constance ( 0 km...,[1036],[]


In [1313]:
def change_subject_relation(factsDF, isQuestion = True):
    if not isQuestion:
        factsDF = factsDF[~((factsDF["Subject"] == "Unknown") & (factsDF["Objects"].apply(len) == 0) & (factsDF["States"].apply(len) == 0) & (factsDF["Times"].apply(len) == 0) & (factsDF["Locations"].apply(len) == 0))]
        factsDF = factsDF.reset_index(drop=True)

    for index, row in factsDF.iterrows():
        factsDF.loc[index, "Subject"] = [row['Subject']]
        factsDF.loc[index, "Relation"] = [row['Relation']]
    return factsDF

In [1314]:
newFactsDF = change_subject_relation(factsDF, False)
#save in csv
newFactsDF.to_csv("facts2.csv")

In [1315]:
newFactsDF

,Subject,Relation,Objects,States,Times,Locations
0,[Unknown],[measure],[The length of the Rhine],[in Rhine-kilometers (Rheinkilometer )],[],[the Rhine]
1,[Unknown],[shorten],[The river length],"[from the river's natural course, due to numbe...",[],[]
2,[canalisation projects],[complete],[],[in the 19th and 20th century.[note 7],[],[]
3,[scale],[introduce],[],[in 1939],[1939],[]
4,[the Alpine Rhine],[be],"[more difficult, to measure objectively ]",[],[],"[the Rhine, the Alpine Rhine]"
5,[the Dutch Rijkswaterstaat],[cite],[it],"[as 1,232 kilometres ( 766 miles ), in 2010.[n...",[2010],[]
6,[the Rhine],[measure],[],[],[],[]
7,[which],[run],[],[from the Old Rhine Bridge at Constance ( 0 km...,[1036],[]


In [1316]:
newQuestionDF = change_subject_relation(questionDF, False)
newQuestionDF

,Subject,Relation,Objects,States,Times,Locations
0,[the measurement of the Rhine],[introduce],[],[],[],[the Rhine]


In [1317]:
def similarity(factRow, questionRow, column):
    if len(factRow[column]) == 0 or len(questionRow[column]) == 0 or factRow[column] == ["Unknown"] or questionRow[column] == ["Unknown"]:
        return 0
    columnString = " ".join(factRow[column])
    questionString = " ".join(questionRow[column])
    embeddingFact = model.encode(columnString)
    embeddingQuestion = model.encode(questionString)
    return util.cos_sim(embeddingFact, embeddingQuestion)

In [1318]:
def cost_function(factsDf, questionFact, excludeColumns=[]):
    score = 0
    maxFactIdx = 0
    columnNames = ["Subject","Relation", "Objects", "States", "Times", "Locations"]
    for column in excludeColumns:
        columnNames.remove(column)
    for factIdx, factRow in factsDf.iterrows():
        currScore = 0
        for _, questionRow in questionFact.iterrows():
            if len(factRow[excludeColumns[0]]) == 0:
                continue
            for column in columnNames:
                currScore += similarity(factRow, questionRow, column)
        if currScore > score:
            score = currScore
            maxFactIdx = factIdx
    return maxFactIdx, score
        

In [1319]:
newQuestionDF

,Subject,Relation,Objects,States,Times,Locations
0,[the measurement of the Rhine],[introduce],[],[],[],[the Rhine]


In [1320]:
correctIdx, _ = cost_function(newFactsDF, newQuestionDF, excludeColumns=[excludesPerQuestionType[question_type]])
WhenAnswer = newFactsDF.loc[correctIdx, excludesPerQuestionType[question_type]]
if WhenAnswer == []:
    WhenAnswer = newFactsDF.loc[correctIdx, "States"]    
" ".join(WhenAnswer)

'1939'

In [1321]:
model = SentenceTransformer("all-MiniLM-L6-v2")

c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [1322]:
emb1 = model.encode("did Lev's Stadium open")
emb2 = model.encode("open")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim.item())

Cosine-Similarity: 0.2557494044303894


In [1323]:
# 0.14 + 0.38
# 

In [1324]:
nlp.get_pipe("ner").labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')